In [4]:
# General libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Libraries that are used for training the model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.kernel_approximation import RBFSampler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_approximation import RBFSampler

# Clean and transformd data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# metrics
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

See the whole size of each dataset

In [5]:
real1 = pd.read_csv('../data/real1/snv-parse-real1-labeled.txt', sep='\t', dtype={'Chr': str})
real1['Chr'] = real1['Chr'].astype('str') 
len(real1)

49320

In [6]:
real2 = pd.read_csv('../data/real2_part1/snv-parse-real2_part1-labeled.txt', sep='\t', dtype={'Chr': str})
real2['Chr'] = real2['Chr'].astype('str') 
len(real2)

22600

In [7]:
syn1 = pd.read_csv('../data/syn1/snv-parse-syn1-labeled.txt', sep='\t', dtype={'Chr': str})
syn1['Chr'] = syn1['Chr'].astype('str') 
len(syn1)

47880

In [8]:
syn2 = pd.read_csv('../data/syn2/snv-parse-syn2-labeled.txt', sep='\t', dtype={'Chr': str})
syn2['Chr'] = syn2['Chr'].astype('str') 
len(syn2)

45376

In [9]:
syn3 = pd.read_csv('../data/syn3/snv-parse-syn3-labeled.txt', sep='\t', dtype={'Chr': str})
syn3['Chr'] = syn3['Chr'].astype('str') 
len(syn3)

44926

In [10]:
syn4 = pd.read_csv('../data/syn4/snv-parse-syn4-labeled.txt', sep='\t', dtype={'Chr': str})
syn4['Chr'] = syn4['Chr'].astype('str') 
len(syn4)

49884

In [11]:
syn5 = pd.read_csv('../data/syn5/snv-parse-syn5-labeled.txt', sep='\t', dtype={'Chr': str})
syn5['Chr'] = syn5['Chr'].astype('str') 
len(syn5)

46235

Print the size of the total dataset

In [12]:
(len(real1))+(len(real2))+(len(syn1))+(len(syn2))+(len(syn3))+(len(syn4))+(len(syn5))

306221

See the 70% that will be used for the training set

In [13]:
((len(real1))+(len(real2))+(len(syn1))+(len(syn2))+(len(syn3))+(len(syn4))+(len(syn5))) * .7

214354.69999999998

The datasets below will be the ones to be used for training the model. To prevent overfitting and having the best accurate model, we will only be performing the training on the next four datasets. With this we will have our own training, validation, and testing datasets. After obtaining the model, we will compare the results with the other datasets. 

In [14]:
(len(real1))+(len(syn1))+(len(syn2))+(len(syn4))+(len(syn5))

238695

In [15]:
dataset = pd.concat([real1, syn1, syn2, syn5], ignore_index=True)
len(dataset)

188811

In [16]:
dataset.head()

,Chr,START_POS_REF,END_POS_REF,REF,ALT,REF_MFVdVs,ALT_MFVdVs,Sample_Name,FILTER_Mutect2,FILTER_Freebayes,FILTER_Vardict,FILTER_Varscan,m2_MQ,f_MQMR,vs_SSC,vs_SPV,vd_SSF,vd_MSI,True_SNV
0,1,13110,13110,G,A,G/NA/G/G/,A/NA/A/A/,icgc_cll-T,True,False,False,False,41.91,NaN,2.0,0.522430,0.23427,2.0,False
1,1,15015,15015,G,C,G/NA/NA/G/,C/NA/NA/C/,icgc_cll-T,True,False,False,False,43.42,NaN,5.0,0.302390,NaN,NaN,False
2,1,16949,16949,A,C,NA/NA/NA/A/,NA/NA/NA/C/,icgc_cll-T,False,False,False,True,NaN,NaN,16.0,0.023282,NaN,NaN,False
3,1,40552,40552,T,C,NA/NA/NA/T/,NA/NA/NA/C/,icgc_cll-T,False,False,False,True,NaN,NaN,26.0,0.002231,NaN,NaN,False
4,1,46907,46907,T,C,NA/NA/NA/T/,NA/NA/NA/C/,icgc_cll-T,False,False,False,True,NaN,NaN,17.0,0.017670,NaN,NaN,False


Count all the rows that have a null value. As we can see, m2_mQ have similar amount of missing values. Then cv_SSC and vs_SPV similar amount of null values, and vd_SSF with vs_MSI have the same amount of null values. 

In [17]:
dataset.isnull().sum()

Chr                      0
START_POS_REF            0
END_POS_REF              0
REF                      0
ALT                      0
REF_MFVdVs               0
ALT_MFVdVs               0
Sample_Name              0
FILTER_Mutect2           0
FILTER_Freebayes         0
FILTER_Vardict           0
FILTER_Varscan           0
m2_MQ               107339
f_MQMR              116574
vs_SSC               36951
vs_SPV               36951
vd_SSF               90405
vd_MSI               90405
True_SNV                 0
dtype: int64

# Train the model with only the SNV callers

The next step is we are going to just take the columsn of fiter mutec2, frebayes, vardict, and varscan to train the model. Also, we will include the results.

In [18]:
columns_to_read = ["FILTER_Mutect2", "FILTER_Freebayes", "FILTER_Vardict", "FILTER_Varscan", "True_SNV"]
only_SNV_callers = dataset[columns_to_read]
only_SNV_callers.head()

,FILTER_Mutect2,FILTER_Freebayes,FILTER_Vardict,FILTER_Varscan,True_SNV
0,True,False,False,False,False
1,True,False,False,False,False
2,False,False,False,True,False
3,False,False,False,True,False
4,False,False,False,True,False


Separate the dataset only in features and results. 

In [19]:
X = only_SNV_callers.iloc[:, :-1].values
y = only_SNV_callers.iloc[:, -1].values

Separate the dataset into training set and testing set. 

### Train Linear SVC
Train the model Linear SVC

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 0, stratify=y)
classifier_linearSVC = LinearSVC(random_state=0, dual=False)  # `dual=False` improves performance for large datasets
classifier_linearSVC.fit(X_train, y_train)

y_pred = classifier_linearSVC.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1

0.9635791068177892

### Train traditional SVM

In [21]:
# Use only 10% of the dataset to train because if not then the SVM will take too long
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.90, random_state = 0, stratify=y)

classifier_SVC = SVC(kernel='linear', random_state=0) # classic model with the linear kernel
classifier_SVC.fit(X_train, y_train)

y_pred = classifier_SVC.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1

0.962877030162413

### Stochastic Gradient Descent Classifier

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 0, stratify=y)

from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler

# Approximate the RBF kernel
rbf_feature = RBFSampler(gamma=1, random_state=0)
X_train_rbf = rbf_feature.fit_transform(X_train)
X_test_rbf = rbf_feature.transform(X_test)

# Train with SGDClassifier (approximate SVM with RBF)
sgd_rbf = SGDClassifier(loss='hinge', random_state=0)
sgd_rbf.fit(X_train_rbf, y_train)

# Predict
y_pred = sgd_rbf.predict(X_test_rbf)
f1 = f1_score(y_test, y_pred)
f1

0.9635791068177892

### Kernel Approximation + Logistic Regression

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 0, stratify=y)

from sklearn.linear_model import LogisticRegression
from sklearn.kernel_approximation import RBFSampler

# RBF kernel approximation
rbf_feature = RBFSampler(gamma=1, random_state=0)
X_train_rbf = rbf_feature.fit_transform(X_train)
X_test_rbf = rbf_feature.transform(X_test)

# Train logistic regression
log_reg = LogisticRegression()
log_reg.fit(X_train_rbf, y_train)

# Predict
y_pred = log_reg.predict(X_test_rbf)
f1 = f1_score(y_test, y_pred)
f1

0.9635791068177892

### Random Fourier Features

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 0, stratify=y)

from sklearn.kernel_approximation import RBFSampler
from sklearn.svm import LinearSVC

# Approximate RBF kernel using Fourier Features
rbf_feature = RBFSampler(gamma=1, random_state=0)
X_train_rbf = rbf_feature.fit_transform(X_train)
X_test_rbf = rbf_feature.transform(X_test)

# Train LinearSVC on transformed features
classifier_rbf_approx = LinearSVC(random_state=0)
classifier_rbf_approx.fit(X_train_rbf, y_train)

# Predict
y_pred = log_reg.predict(X_test_rbf)
f1 = f1_score(y_test, y_pred)
f1

0.9635791068177892

# Train the model with SNV callers + 6 other features

In [30]:
dataset = pd.concat([real1, syn1, syn2, syn5], ignore_index=True)
columns_to_read = ["FILTER_Mutect2", "FILTER_Freebayes", "FILTER_Vardict", "FILTER_Varscan", "m2_MQ", "f_MQMR", "vs_SSC", "vs_SPV", "vd_SSF", "vd_MSI", "True_SNV"]
only_SNV_callers = dataset[columns_to_read]
only_SNV_callers.head()

,FILTER_Mutect2,FILTER_Freebayes,FILTER_Vardict,FILTER_Varscan,m2_MQ,f_MQMR,vs_SSC,vs_SPV,vd_SSF,vd_MSI,True_SNV
0,True,False,False,False,41.91,NaN,2.0,0.522430,0.23427,2.0,False
1,True,False,False,False,43.42,NaN,5.0,0.302390,NaN,NaN,False
2,False,False,False,True,NaN,NaN,16.0,0.023282,NaN,NaN,False
3,False,False,False,True,NaN,NaN,26.0,0.002231,NaN,NaN,False
4,False,False,False,True,NaN,NaN,17.0,0.017670,NaN,NaN,False


The code belows does this: 
1. For each continous column, get the 25%, 50%, and 75% of the total column
2. Replaces each continious value for 4 categorical values, if they are in the range 0-25%, 25-50%, 50-75%, or >75%.
3. The Nan values are replace for zeros

In [31]:
whole_dataset = pd.concat([real1, real2, syn1, syn2, syn3, syn4, syn5], ignore_index=True)
whole_dataset = whole_dataset[columns_to_read]

In [32]:
# return a dataframe where the vlues of continous columns where transformed to categorical
continuous_columns = ["m2_MQ", "f_MQMR", "vs_SSC", "vs_SPV", "vd_SSF", "vd_MSI"]
def transform_continous_cols_in(dataset):
    # Define the continuous variables that need to be categorized
    def transform_column(name_col, dataset):
        q25 = whole_dataset.describe()[name_col]['25%']
        q50 = whole_dataset.describe()[name_col]['50%']
        q75 = whole_dataset.describe()[name_col]['75%']

        def return_new_(value):
            # return 1 # just return 1 if it has a value
            if value < q25:
                return int(1)
            elif value < q50:
                return int(2)
            elif value < q75:
                return int(3)
            else:
                return int(4)
    
        return dataset[name_col].map(return_new_, na_action='ignore')

    new_dataset = dataset.copy()
    # transfrom each columns continious values
    for column in continuous_columns:
        new_dataset[column] = transform_column(column, new_dataset)
        
    new_dataset.fillna(0, inplace=True) # transform all nan values
    new_dataset[continuous_columns] = new_dataset[continuous_columns].astype(int) # transform the column to integer
    return new_dataset

In [33]:
whole_dataset2 = whole_dataset[columns_to_read]
whole_dataset2 = transform_continous_cols_in(whole_dataset)

def transform_using_col_transformer(dataset):
    dataset_transformed = transform_continous_cols_in(dataset.copy())
    global_column_transformer = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), continuous_columns)], remainder='passthrough')
    gct = global_column_transformer.fit(whole_dataset2)
    transformed_dataset = gct.transform(dataset_transformed)
    
    # separate data into x, y values
    X = transformed_dataset[:, :-1]
    y = transformed_dataset[:, -1]
    
    return X, y

## Train SVC + SVM models

In [34]:
X, y = transform_using_col_transformer(only_SNV_callers)
# Linear SVC
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 0, stratify=y)
classifier_linearSVC = LinearSVC(random_state=0, dual=False)  # `dual=False` improves performance for large datasets
classifier_linearSVC.fit(X_train, y_train)

# Traditinal linear svc
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.90, random_state = 0, stratify=y)
classifier_SVC = SVC(kernel='linear', random_state=0) # classic model with the linear kernel
classifier_SVC.fit(X_train, y_train)
print()

### Stochastic Gradient Descent Classifier

In [35]:
X, y = transform_using_col_transformer(only_SNV_callers)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 0, stratify=y)

# Approximate the RBF kernel
rbf_feature = RBFSampler(gamma=1, random_state=0)
X_train_rbf = rbf_feature.fit_transform(X_train)
X_test_rbf = rbf_feature.transform(X_test)

# Train with SGDClassifier (approximate SVM with RBF)
sgd_rbf = SGDClassifier(loss='hinge', random_state=0)
sgd_rbf.fit(X_train_rbf, y_train)

SGDClassifier(random_state=0)

### Kernel Approximation + Logistic Regression

In [36]:
X, y = transform_using_col_transformer(only_SNV_callers)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 0, stratify=y)

# RBF kernel approximation
rbf_feature = RBFSampler(gamma=1, random_state=0)
X_train_rbf = rbf_feature.fit_transform(X_train)
X_test_rbf = rbf_feature.transform(X_test)

# Train logistic regression
log_reg = LogisticRegression()
log_reg.fit(X_train_rbf, y_train)



LogisticRegression()

### Random Fourier Features

In [37]:
X, y = transform_using_col_transformer(only_SNV_callers)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 0, stratify=y)

# Approximate RBF kernel using Fourier Features
rbf_feature = RBFSampler(gamma=1, random_state=0)
X_train_rbf = rbf_feature.fit_transform(X_train)
X_test_rbf = rbf_feature.transform(X_test)

# Train LinearSVC on transformed features
classifier_rbf_approx = LinearSVC(random_state=0)
classifier_rbf_approx.fit(X_train_rbf, y_train)

LinearSVC(random_state=0)

# Testing the models for Four Variables
We will all the datasets to see how well the model performed.

In [25]:
def test_model(dataset, dataset_name, model, model_name):
    # Filter to have only the columns that matter
    columns_to_read = ["FILTER_Mutect2", "FILTER_Freebayes", "FILTER_Vardict", "FILTER_Varscan", "True_SNV"]
    current_dataset = dataset[columns_to_read]
    
    # Divide the data in X, y matrices
    local_X = current_dataset.iloc[:, :-1].values
    local_y = current_dataset.iloc[:, -1].values
    
    # Get the prediction
    y_pred = model.predict(local_X)
    
    # Check the f1 score
    f1 = f1_score(local_y, y_pred)
    print(f'The model {model_name} performed in {dataset_name} dataset has an f1 score of: ', f1)

Datasets that will be used to test the model

In [26]:
datasets = [real1, real2, syn1, syn2, syn3, syn4, syn5]
dataset_names = ['rea1', 'real2', 'syn1', 'syn2', 'syn3', 'syn4', 'syn5']

## Testing linear SVMs

In [27]:
models = [classifier_linearSVC, classifier_SVC]
models_names = ['classifier_linearSVC', 'classifier_SVC']

for model, model_name in zip(models, models_names): 
    for dataset, dataset_name in zip(datasets, dataset_names):
        test_model(dataset, dataset_name, model, model_name)
    print()

The model classifier_linearSVC performed in rea1 dataset has an f1 score of:  0.8089401586157173
The model classifier_linearSVC performed in real2 dataset has an f1 score of:  0.6488352027610008
The model classifier_linearSVC performed in syn1 dataset has an f1 score of:  0.9077771939043615
The model classifier_linearSVC performed in syn2 dataset has an f1 score of:  0.9042382833351126
The model classifier_linearSVC performed in syn3 dataset has an f1 score of:  0.9608814708566958
The model classifier_linearSVC performed in syn4 dataset has an f1 score of:  0.8931939469259449
The model classifier_linearSVC performed in syn5 dataset has an f1 score of:  0.9804949783974191

The model classifier_SVC performed in rea1 dataset has an f1 score of:  0.8265961465810351
The model classifier_SVC performed in real2 dataset has an f1 score of:  0.7011278195488722
The model classifier_SVC performed in syn1 dataset has an f1 score of:  0.9216664419576648
The model classifier_SVC performed in syn2 da

## Testing other SVMs

Testing on the model using random fourier features

In [28]:
def test_model_rbf(dataset, dataset_name, model, model_name):
    # Filter to have only the columns that matter
    columns_to_read = ["FILTER_Mutect2", "FILTER_Freebayes", "FILTER_Vardict", "FILTER_Varscan", "True_SNV"]
    current_dataset = dataset[columns_to_read]
    
    # Divide the data in X, y matrices
    local_X = current_dataset.iloc[:, :-1].values
    local_y = current_dataset.iloc[:, -1].values

    # RBF kernel approximation
    rbf_feature = RBFSampler(gamma=1, random_state=0)
    X_rbf = rbf_feature.fit_transform(local_X)
    
    # Get the prediction
    y_pred = model.predict(X_rbf)
    
    # Check the f1 score
    f1 = f1_score(local_y, y_pred)
    print(f'The model {model_name} performed in {dataset_name} dataset has an f1 score of: ', f1)

In [29]:
models = [sgd_rbf, log_reg, classifier_rbf_approx ]
models_names = ['SGDClassifier', 'rbf_feature', 'classifier_rbf_approx ']

for model, model_name in zip(models, models_names): 
    for dataset, dataset_name in zip(datasets, dataset_names):
        test_model_rbf(dataset, dataset_name, model, model_name)
    print()

The model SGDClassifier performed in rea1 dataset has an f1 score of:  0.8089401586157173
The model SGDClassifier performed in real2 dataset has an f1 score of:  0.6488352027610008
The model SGDClassifier performed in syn1 dataset has an f1 score of:  0.9077771939043615
The model SGDClassifier performed in syn2 dataset has an f1 score of:  0.9042382833351126
The model SGDClassifier performed in syn3 dataset has an f1 score of:  0.9608814708566958
The model SGDClassifier performed in syn4 dataset has an f1 score of:  0.8931939469259449
The model SGDClassifier performed in syn5 dataset has an f1 score of:  0.9804949783974191

The model rbf_feature performed in rea1 dataset has an f1 score of:  0.8089401586157173
The model rbf_feature performed in real2 dataset has an f1 score of:  0.6488352027610008
The model rbf_feature performed in syn1 dataset has an f1 score of:  0.9077771939043615
The model rbf_feature performed in syn2 dataset has an f1 score of:  0.9042382833351126
The model rbf_f

# Testing the model with variant callers quality controls

In [38]:
def test_model(dataset, dataset_name, model, model_name):
    # Filter to have only the columns that matter
    columns_to_read = ["FILTER_Mutect2", "FILTER_Freebayes", "FILTER_Vardict", "FILTER_Varscan", "m2_MQ", "f_MQMR", "vs_SSC", "vs_SPV", "vd_SSF", "vd_MSI", "True_SNV"]
    current_dataset = dataset[columns_to_read]
    
    # Divide the data in X, y matrices
    local_X, local_y = transform_using_col_transformer(current_dataset)
    # Get the prediction
    y_pred = model.predict(local_X)
    
    # Check the f1 score
    f1 = f1_score(local_y, y_pred)
    print(f'The model {model_name} performed in {dataset_name} dataset has an f1 score of: ', f1)

datasets = [real1, real2, syn1, syn2, syn3, syn4, syn5]
dataset_names = ['rea1', 'real2', 'syn1', 'syn2', 'syn3', 'syn4', 'syn5']

## Testing Linear SVMs

In [39]:
models = [classifier_linearSVC, classifier_SVC]
models_names = ['classifier_linearSVC', 'classifier_SVC']

for model, model_name in zip(models, models_names): 
    for dataset, dataset_name in zip(datasets, dataset_names):
        test_model(dataset, dataset_name, model, model_name)
    print()

The model classifier_linearSVC performed in rea1 dataset has an f1 score of:  0.8541133455210238
The model classifier_linearSVC performed in real2 dataset has an f1 score of:  0.7605633802816901
The model classifier_linearSVC performed in syn1 dataset has an f1 score of:  0.9162913907284769
The model classifier_linearSVC performed in syn2 dataset has an f1 score of:  0.9060726143300846
The model classifier_linearSVC performed in syn3 dataset has an f1 score of:  0.9646717637753757
The model classifier_linearSVC performed in syn4 dataset has an f1 score of:  0.8844886931418632
The model classifier_linearSVC performed in syn5 dataset has an f1 score of:  0.9816790590164308

The model classifier_SVC performed in rea1 dataset has an f1 score of:  0.8527245949926362
The model classifier_SVC performed in real2 dataset has an f1 score of:  0.7803347280334728
The model classifier_SVC performed in syn1 dataset has an f1 score of:  0.9195617316942811
The model classifier_SVC performed in syn2 da

## Testing other SVMs
Testing the models using random fourier features

In [40]:
def test_model_rbf(dataset, dataset_name, model, model_name):
    # Filter to have only the columns that matter
    columns_to_read = ["FILTER_Mutect2", "FILTER_Freebayes", "FILTER_Vardict", "FILTER_Varscan", "m2_MQ", "f_MQMR", "vs_SSC", "vs_SPV", "vd_SSF", "vd_MSI", "True_SNV"]
    current_dataset = dataset[columns_to_read]
    
    # Divide the data in X, y matrices
    local_X, local_y = transform_using_col_transformer(current_dataset)

    # RBF kernel approximation
    rbf_feature = RBFSampler(gamma=1, random_state=0)
    X_rbf = rbf_feature.fit_transform(local_X)
    
    # Get the prediction
    y_pred = model.predict(X_rbf)
    
    # Check the f1 score
    f1 = f1_score(local_y, y_pred)
    print(f'The model {model_name} performed in {dataset_name} dataset has an f1 score of: ', f1)

In [41]:
models = [sgd_rbf, log_reg, classifier_rbf_approx ]
models_names = ['SGDClassifier', 'rbf_feature', 'classifier_rbf_approx ']

for model, model_name in zip(models, models_names): 
    for dataset, dataset_name in zip(datasets, dataset_names):
        test_model_rbf(dataset, dataset_name, model, model_name)
    print()

The model SGDClassifier performed in rea1 dataset has an f1 score of:  0.5674246430460074
The model SGDClassifier performed in real2 dataset has an f1 score of:  0.39932697700504766
The model SGDClassifier performed in syn1 dataset has an f1 score of:  0.7502903600464577
The model SGDClassifier performed in syn2 dataset has an f1 score of:  0.7696493349455864
The model SGDClassifier performed in syn3 dataset has an f1 score of:  0.8303707130112998
The model SGDClassifier performed in syn4 dataset has an f1 score of:  0.7444998533294221
The model SGDClassifier performed in syn5 dataset has an f1 score of:  0.9403140617228394

The model rbf_feature performed in rea1 dataset has an f1 score of:  0.5416872224963
The model rbf_feature performed in real2 dataset has an f1 score of:  0.3869565217391304
The model rbf_feature performed in syn1 dataset has an f1 score of:  0.7357928505957837
The model rbf_feature performed in syn2 dataset has an f1 score of:  0.7548691717489672
The model rbf_fea